In [1]:
import torch
import torch.nn as nn
from random import sample
import sys
sys.path.append("/home/siyi/project/PCL/pcl")
from head import Yolov8Head, MlpHead
from backbone import resnet50

In [3]:
channels = 1024  # 这是 q_dense 的通道维度
height, width = 7, 7  # 这是 q_dense 的空间维度
r = 4

In [4]:
queue2 = torch.randn(channels, r, height, width)
queue2 = nn.functional.normalize(queue2, dim=0)

In [5]:
queue2.detach().shape

torch.Size([1024, 4, 7, 7])

In [6]:
queue2_ptr = torch.zeros(1, dtype=torch.long)

In [ ]:
l_global_neg = torch.einsum('nc,ck->nk', [q_global, queue.clone().detach()])

In [20]:
# 通过emd计算相似度
x1 = torch.randn(1, 1024, 7, 7)
x2 = torch.randn(1, 1024, 7, 7)

In [11]:
def sinkhorn_knopp(a, b, cost_matrix, epsilon, max_iter=100):
    """
    Compute Sinkhorn distance (approximation of EMD) between distributions a and b
    with regularization parameter epsilon.
    """
    n, m = cost_matrix.shape
    u = torch.ones(n, dtype=torch.float32, device=a.device) / n
    v = torch.ones(m, dtype=torch.float32, device=b.device) / m

    K = torch.exp(-cost_matrix / epsilon)

    for _ in range(max_iter):
        u = a / (K @ v)
        v = b / (K.t() @ u)

    transport_plan = torch.diag(u) @ K @ torch.diag(v)
    distance = torch.sum(transport_plan * cost_matrix)

    return distance, transport_plan


def compute_emd_torch(x, y, epsilon=1e-3, max_iter=100):
    """
    Compute the EMD between two distributions using Sinkhorn-Knopp algorithm.
    x, y should be histograms or distributions.
    """
    assert x.shape == y.shape, "Input distributions must have the same shape."
    n = x.shape[0]

    cost_matrix = torch.cdist(x.unsqueeze(0), y.unsqueeze(0), p=2).squeeze(0)
    print("cost_matrix.shape:", cost_matrix.shape)
    a = torch.ones(n, dtype=torch.float32, device=x.device) / n
    b = torch.ones(n, dtype=torch.float32, device=y.device) / n

    emd, transport_plan = sinkhorn_knopp(a, b, cost_matrix, epsilon, max_iter)

    return emd.item()

In [12]:
compute_emd_torch(x1, x2)

cost_matrix.shape: torch.Size([1024, 7, 7])


ValueError: too many values to unpack (expected 2)

In [13]:
class SinkhornDistance(torch.nn.Module):
    def __init__(self, eps, max_iter, reduction="mean"):
        super(SinkhornDistance, self).__init__()
        self.eps = eps
        self.max_iter = max_iter
        self.reduction = reduction

    def forward(self, x, y):
        # 扁平化空间维度
        batch_size, num_points, height, width = x.size()
        x = x.view(batch_size, num_points, -1)
        y = y.view(batch_size, num_points, -1)

        # 归一化特征向量
        x = F.normalize(x, p=2, dim=-1)
        y = F.normalize(y, p=2, dim=-1)

        # 计算余弦相似度矩阵并归一化
        cost_matrix = 1 - F.cosine_similarity(x.unsqueeze(2), y.unsqueeze(1), dim=-1)
        cost_matrix = cost_matrix / cost_matrix.max()  # 归一化到 [0, 1] 范围内

        # 初始分布
        u = torch.ones(batch_size, num_points).to(x.device) / num_points
        v = torch.ones(batch_size, num_points).to(x.device) / num_points

        # Sinkhorn 迭代
        for _ in range(self.max_iter):
            u = 1.0 / (cost_matrix.bmm(v.unsqueeze(-1)).squeeze(-1) + self.eps)
            v = 1.0 / (
                cost_matrix.transpose(1, 2).bmm(u.unsqueeze(-1)).squeeze(-1) + self.eps
            )

        # 计算Wasserstein距离
        distance = (u.unsqueeze(-1) * cost_matrix * v.unsqueeze(-2)).sum(dim=(1, 2))

        if self.reduction == "mean":
            distance = distance.mean()
        elif self.reduction == "sum":
            distance = distance.sum()

        return distance

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
distance = SinkhornDistance(eps=0.5, max_iter=100, reduction="mean")

In [25]:
distance(x1, x2)

tensor(1004.6594)

In [26]:
x3 = torch.rand(1, 1024, 7, 7)

In [27]:
distance(x1, x3)

tensor(1004.5660)

In [28]:
import torch
import torch.nn.functional as F

class SinkhornDistance(torch.nn.Module):
    def __init__(self, eps=1e-3, max_iter=100, reduction='mean'):
        super(SinkhornDistance, self).__init__()
        self.eps = eps
        self.max_iter = max_iter
        self.reduction = reduction

    def forward(self, x, y):
        # 扁平化空间维度
        batch_size, num_points, height, width = x.size()
        x = x.view(batch_size, num_points, -1)
        y = y.view(batch_size, num_points, -1)

        # 归一化特征向量到概率分布
        x = F.softmax(x, dim=-1)
        y = F.softmax(y, dim=-1)

        # 计算 Euclidean 距离作为成本矩阵
        cost_matrix = torch.cdist(x, y, p=2)

        # 初始分布
        u = torch.ones(batch_size, num_points).to(x.device) / num_points
        v = torch.ones(batch_size, num_points).to(x.device) / num_points

        # Sinkhorn 迭代
        for _ in range(self.max_iter):
            u = self.eps / (cost_matrix.bmm(v.unsqueeze(-1)).squeeze(-1) + self.eps)
            v = self.eps / (cost_matrix.transpose(1, 2).bmm(u.unsqueeze(-1)).squeeze(-1) + self.eps)

        # 计算 Wasserstein 距离
        distance = (u.unsqueeze(-1) * cost_matrix * v.unsqueeze(-2)).sum(dim=(1, 2))

        if self.reduction == 'mean':
            distance = distance.mean()
        elif self.reduction == 'sum':
            distance = distance.sum()

        return distance

# 示例使用
x1 = torch.randn(1, 1024, 7, 7)
x2 = torch.randn(1, 1024, 7, 7)
sinkhorn = SinkhornDistance()
distance = sinkhorn(x1, x2)
print(distance)

tensor(1.0227)


In [1]:
import torch

In [2]:
q_dense = torch.rand(20, 1024, 7, 7)

In [3]:
q_dense_1 = q_dense[1]

In [5]:
q_dense_1.unsqueeze(0).shape

torch.Size([1, 1024, 7, 7])